# Image Preprocess

In [ ]:
import sys
from pathlib import Path

PROJECT_ROOT = Path().resolve().parents[1]
if str(PROJECT_ROOT) not in sys.path:
	sys.path.insert(0, str(PROJECT_ROOT))

import cv2 as cv
import numpy as np

from Code.image import ImgPreproc, ImgPreprocCfg

Imgcfg = ImgPreprocCfg(
	target_size=512,
	sigma=2.0,
	flag_refine_mask=True,
	open_ksize=5,
	close_ksize=5,
)
pre = ImgPreproc(cfg=Imgcfg)


run = 19
dataset = 6

# --- Configuración de paths ---
in_path = PROJECT_ROOT / "Database" / "data" / f"image{dataset}"

out_path = PROJECT_ROOT / "Database" / "tmp" / "image" / f"Preprocess Intento {run:02d}"
out_path.mkdir(parents=True, exist_ok=True)
labels = ["Clavo", "Tornillo", "Tuerca", "Arandela"]

for l in labels:
	input_dir = in_path / l
	out_dir = out_path / l
	out_dir.mkdir(parents=True, exist_ok=True)

paths = list(in_path.glob("*/*.jpg"))

for p in paths:
	i_bgr = cv.imread(str(p))
	if i_bgr is None:
		continue

	img_sq, mask_sq = pre.procesar(i_bgr, blacknwhite=False)

	out_dir = out_path / p.parent.name
	out_dir.mkdir(parents=True, exist_ok=True)

	cv.imwrite(str(out_dir / f"{p.stem} recortada.png"), img_sq)
	cv.imwrite(str(out_dir / f"{p.stem} mask.png"), mask_sq)




# Image Features

In [ ]:
import sys
from pathlib import Path

PROJECT_ROOT = Path().resolve().parents[1]
if str(PROJECT_ROOT) not in sys.path:
	sys.path.insert(0, str(PROJECT_ROOT))

import cv2 as cv
import numpy as np
import pandas as pd

from Code.image import ImgPreproc, ImgPreprocCfg
from Code.image import ImgFeat, hyper_params

Imgcfg = ImgPreprocCfg(
	target_size=512,
	sigma=2.0,
	flag_refine_mask=True,
	open_ksize=5,
	close_ksize=5,
)
pre = ImgPreproc(cfg=Imgcfg)

hp = hyper_params(
	radial_var_t_low=0.0,
	radial_var_t_high=0.045,
	r_hull_t=0.225,
)
feat = ImgFeat(hp=hp, mode="3D")

run = 3
dataset = 6

# --- Configuración de paths ---
in_path = PROJECT_ROOT / "Database" / "data" / f"image{dataset}"

out_path = PROJECT_ROOT / "Database" / "tmp" / "image" / f"Features Intento {run:02d}"
out_path.mkdir(parents=True, exist_ok=True)
labels = ["Clavo", "Tornillo", "Tuerca", "Arandela"]

rows = []
names = None
resultados = []
out_dir = []

for l in labels:
	out = out_path / l
	out.mkdir(parents=True, exist_ok=True)
	out_dir.append(out)

paths = sorted(in_path.glob("*/*.jpg"))

for p in paths:
	i_bgr = cv.imread(str(p))
	if i_bgr is None:
		continue

	img_sq, mask_sq = pre.procesar(i_bgr, blacknwhite=False)

	out = out_path / p.parent.name
	out.mkdir(parents=True, exist_ok=True)

	cv.imwrite(str(out / f"{p.stem} recortada.png"), img_sq)
	cv.imwrite(str(out / f"{p.stem} mask.png"), mask_sq)

	label = p.parent.name
	vec, names, debug = feat.extraer_features(img_sq, mask_sq)
	rows.append([label, p.name, *vec.tolist()])
	resultados.append(vec)

vec_alphas = [1]

X = np.asarray(resultados)
df_meta = pd.DataFrame(rows, columns=["clase", "archivo", *names])[["clase", "archivo"]]

for alpha in vec_alphas:
	X_scaled = X.copy()
	X_scaled[:, 2] *= alpha

	df_base = pd.DataFrame(X_scaled, columns=names)
	df_out = pd.concat([df_meta.reset_index(drop=True), df_base], axis=1)

	csv_path = out_path / f"features_alpha{alpha}.csv"
	df_out.to_csv(csv_path, index=False)
	display(df_out)




# K Means Model Tryout

In [ ]:
import sys
from pathlib import Path

PROJECT_ROOT = Path().resolve().parents[1]
if str(PROJECT_ROOT) not in sys.path:
	sys.path.insert(0, str(PROJECT_ROOT))

import cv2 as cv
import numpy as np
import pandas as pd

from Code.image import ImgPreproc, ImgPreprocCfg
from Code.image import ImgFeat, hyper_params
from Code.image import KMeansModel

Imgcfg = ImgPreprocCfg(
	target_size=512,
	sigma=2.0,
	flag_refine_mask=True,
	open_ksize=5,
	close_ksize=5,
)
pre = ImgPreproc(cfg=Imgcfg)

hp = hyper_params(
	radial_var_t_low=0.0,
	radial_var_t_high=0.045,
	r_hull_t=0.225,
)
feat = ImgFeat(hp=hp, mode="3D")

semilla_ = np.array([
	[1.0, 0.0, 0.0],    # Arandela
	[0.0, 0.0, 1.0],    # Clavo
	[0.0, 1.0, 1.0],    # Tornillo
	[1.0, 0.0, 1.0],    # Tuerca
], dtype=float)

model = KMeansModel(n_clusters=4, init_centers=semilla_)

run = 3
dataset = 6

# --- Configuración de paths ---
in_path = PROJECT_ROOT / "Database" / "data" / f"image{dataset}"

out_path = PROJECT_ROOT / "Database" / "tmp" / "image" / f"Model Intento {run:02d}"
out_path.mkdir(parents=True, exist_ok=True)
labels = ["Clavo", "Tornillo", "Tuerca", "Arandela"]

rows = []
names = None
resultados = []
out_dir = []

for l in labels:
	out = out_path / l
	out.mkdir(parents=True, exist_ok=True)
	out_dir.append(out)

paths = sorted(in_path.glob("*/*.jpg"))

for p in paths:
	i_bgr = cv.imread(str(p))
	if i_bgr is None:
		continue

	img_sq, mask_sq = pre.procesar(i_bgr, blacknwhite=False)

	out = out_path / p.parent.name
	out.mkdir(parents=True, exist_ok=True)

	cv.imwrite(str(out / f"{p.stem} recortada.png"), img_sq)
	cv.imwrite(str(out / f"{p.stem} mask.png"), mask_sq)

	label = p.parent.name
	vec, names, debug = feat.extraer_features(img_sq, mask_sq)
	rows.append([label, p.name, *vec.tolist()])
	resultados.append(vec)

vec_alphas = [1]

X = np.asarray(resultados)
df_meta = pd.DataFrame(rows, columns=["clase", "archivo", *names])[["clase", "archivo"]]

for alpha in vec_alphas:
	X_scaled = X.copy()
	X_scaled[:, 2] *= alpha

	df_base = pd.DataFrame(X_scaled, columns=names)
	df_out = pd.concat([df_meta.reset_index(drop=True), df_base], axis=1)

	csv_path = out_path / f"features_alpha{alpha}.csv"
	df_out.to_csv(csv_path, index=False)
	display(df_out)

model.fit(X)
centers = model._centers
labels = model.predict(X)
inertia = model._inertia

results = [{"seed": "manual", "inertia": inertia, "centers": centers, "labels": labels}]

df_runs = pd.DataFrame([{"seed": r["seed"], "inertia": r["inertia"]} for r in results])

rows = []
for r in results:
	for k, c in enumerate(r["centers"]):
		rows.append({"seed": r["seed"], "cluster": k, **{n: c[i] for i, n in enumerate(names)}})
df_centers = pd.DataFrame(rows)

display(df_runs)
display(df_centers)

# Orchestrator

In [1]:
from pathlib import Path
import sys
import numpy as np

PROJECT_ROOT = Path().resolve().parents[1]
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from Code.image import ImgOrchestrator

# Semillas opcionales
s0 = np.array([
    [1.0, 0.0, 0.0],    # Arandela
    [0.0, 0.0, 1.0],    # Clavo
    [0.0, 1.0, 1.0],    # Tornillo
    [1.0, 0.0, 1.0],    # Tuerca
], dtype=float)

# Entrenar
orch = ImgOrchestrator()
# orch.entrenar(
#     dataset_dir="Database/data/image6",
#     run=1,
#     seeds=s0,
#     output_root="Database/tmp/image/Orchestrator Model Intento 01",
#     labels=["Arandela", "Clavo", "Tornillo", "Tuerca"],
# )

# # Guardar en un solo npz (centroides + mapping)
models_dir = (PROJECT_ROOT / "Database" / "models").resolve()
models_dir.mkdir(parents=True, exist_ok=True)
centroids_path = models_dir / "kmeans.npz"
# orch.guardar_modelo(centroids_path)
# print("Modelo guardado en:", centroids_path)

# (Opcional) inspeccionar
data = np.load(centroids_path, allow_pickle=False)
print("Claves:", data.files)          # ['C', 'cluster_keys', 'cluster_labels']
print("Centroides shape:", data["C"].shape)

display(data["C"])
display(data["cluster_keys"])
display(data["cluster_labels"])

orch.cargar_modelo(centroids_path)

carpeta = PROJECT_ROOT / "Database" / "input" / "image"
resultado, df = orch.predecir(carpeta)  
display(df)

Claves: ['C', 'cluster_keys', 'cluster_labels']
Centroides shape: (4, 3)


array([[1.        , 0.26944762, 0.06787008],
       [0.        , 0.29970637, 1.        ],
       [0.        , 0.8714198 , 1.        ],
       [1.        , 0.26092434, 0.92962277]], dtype=float32)

array([0, 1, 2, 3])

array(['Arandela', 'Clavo', 'Tornillo', 'Tuerca'], dtype='<U64')

,id,path relativo,Número de Cluster,Clasificación
0,1,Arandela01.jpg,0,Arandela
1,2,Arandela02.jpg,0,Arandela
2,3,Clavo01.jpg,1,Clavo
3,4,Clavo02.jpg,1,Clavo
4,5,Tornillo01.jpg,2,Tornillo
5,6,Tornillo02.jpg,2,Tornillo
6,7,Tuerca01.jpg,3,Tuerca
7,8,Tuerca02.jpg,3,Tuerca
